### Import Library

In [1]:
import pandas as pd
import torch

### Input Dataset:

Ep5: https://www.kaggle.com/datasets/xvivancos/star-wars-movie-scripts?resource=download&select=SW_EpisodeV.txt

### Data Inspection

In [2]:
# read it in to inspect it
with open('SW_EpisodeV.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  48009


In [4]:
print(text[:1000])

LUKE: Echo Three to Echo Seven. Han, old buddy, do you read me?
HAN: Loud and clear, kid. What's up?
LUKE: Well, I finished my circle. I don't pick up any life readings.
HAN: There isn't enough life on this ice cube to fill a space cruiser. The sensors are placed. I'm going back.
LUKE: Right. I'll see you shortly. There's a meteorite that hit the ground near here. I want to check it out. It won't take long.
LUKE: Hey, steady girl. What's the matter? You smell something?
LUKE: Aaargh!
HAN: Chewie!
HAN: All right, don't lose your temper. I'll come right back and give  you a hand.
RIEEKAN: Solo?
HAN: No sign of life out there, General. The sensors are in place. You'll know if anything comes around.
RIEEKAN: Commander Skywalker reported in yet?
HAN: No. He's checking out a meteorite that hit near him.
RIEEKAN: With all the meteor activity in this  system, it's going to be difficult to spot approaching ships.
HAN: General, I've got to leave. I can't stay anymore.
RIEEKAN: I'm sorry to hear 

In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"',-.2:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
64


In [6]:
# # create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[45, 46, 46, 1, 57, 45, 42, 55, 42]
hii there


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
# we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([48009]) torch.int64
tensor([23, 32, 22, 16,  9,  1, 16, 40, 45, 52,  1, 31, 45, 55, 42, 42,  1, 57,
        52,  1, 16, 40, 45, 52,  1, 30, 42, 59, 42, 51,  7,  1, 19, 38, 51,  5,
         1, 52, 49, 41,  1, 39, 58, 41, 41, 62,  5,  1, 41, 52,  1, 62, 52, 58,
         1, 55, 42, 38, 41,  1, 50, 42, 11,  0, 19, 12, 25,  9,  1, 23, 52, 58,
        41,  1, 38, 51, 41,  1, 40, 49, 42, 38, 55,  5,  1, 48, 46, 41,  7,  1,
        34, 45, 38, 57,  4, 56,  1, 58, 53, 11,  0, 23, 32, 22, 16,  9,  1, 34,
        42, 49, 49,  5,  1, 20,  1, 43, 46, 51, 46, 56, 45, 42, 41,  1, 50, 62,
         1, 40, 46, 55, 40, 49, 42,  7,  1, 20,  1, 41, 52, 51,  4, 57,  1, 53,
        46, 40, 48,  1, 58, 53,  1, 38, 51, 62,  1, 49, 46, 43, 42,  1, 55, 42,
        38, 41, 46, 51, 44, 56,  7,  0, 19, 12, 25,  9,  1, 31, 45, 42, 55, 42,
         1, 46, 56, 51,  4, 57,  1, 42, 51, 52, 58, 44, 45,  1, 49, 46, 43, 42,
         1, 52, 51,  1, 57, 45, 46, 56,  1, 46, 40, 42,  1, 40, 58, 39, 42,  1,
        

In [8]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
#Split things into smaller chunk
block_size = 8
train_data[:block_size+1]

tensor([23, 32, 22, 16,  9,  1, 16, 40, 45])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1] #next block size (target of the input)
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([23]) the target: 32
when input is tensor([23, 32]) the target: 22
when input is tensor([23, 32, 22]) the target: 16
when input is tensor([23, 32, 22, 16]) the target: 9
when input is tensor([23, 32, 22, 16,  9]) the target: 1
when input is tensor([23, 32, 22, 16,  9,  1]) the target: 16
when input is tensor([23, 32, 22, 16,  9,  1, 16]) the target: 40
when input is tensor([23, 32, 22, 16,  9,  1, 16, 40]) the target: 45


In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[45, 46, 51, 44,  1, 51, 52, 57],
        [51,  4, 57,  1, 39, 42,  1, 52],
        [41, 52,  1, 50, 62,  1, 47, 52],
        [ 5,  1, 46, 43,  1, 62, 52, 58]])
targets:
torch.Size([4, 8])
tensor([[46, 51, 44,  1, 51, 52, 57,  1],
        [ 4, 57,  1, 39, 42,  1, 52, 51],
        [52,  1, 50, 62,  1, 47, 52, 39],
        [ 1, 46, 43,  1, 62, 52, 58,  4]])
----
when input is [45] the target: 46
when input is [45, 46] the target: 51
when input is [45, 46, 51] the target: 44
when input is [45, 46, 51, 44] the target: 1
when input is [45, 46, 51, 44, 1] the target: 51
when input is [45, 46, 51, 44, 1, 51] the target: 52
when input is [45, 46, 51, 44, 1, 51, 52] the target: 57
when input is [45, 46, 51, 44, 1, 51, 52, 57] the target: 1
when input is [51] the target: 4
when input is [51, 4] the target: 57
when input is [51, 4, 57] the target: 1
when input is [51, 4, 57, 1] the target: 39
when input is [51, 4, 57, 1, 39] the target: 42
when input is [51, 4, 

In [12]:
print(xb) # our input to the transformer

tensor([[45, 46, 51, 44,  1, 51, 52, 57],
        [51,  4, 57,  1, 39, 42,  1, 52],
        [41, 52,  1, 50, 62,  1, 47, 52],
        [ 5,  1, 46, 43,  1, 62, 52, 58]])


In [13]:
#Baseline model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

#Note that the output doesn't make sense because at the current point, we feed all but it is only look at the last piece (-1) to predict the next char instead of the whole history

torch.Size([32, 64])
tensor(4.6527, grad_fn=<NllLossBackward0>)

-,!FHAaDUCdiliCcryulI,.ZoODux FS,tvSaD?2R?;gMt
-Czh'?Hal.JL,Zbwi?MvkqYBjZ!dmgBXDkbdS;QFbcR!JS:sXWV'-


In [14]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [22]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.4706151485443115


In [28]:
import numpy as np

A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])

# Using the @ operator for matrix multiplication
C = A @ B

In [29]:
C


array([[19, 22],
       [43, 50]])

In [23]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


RCKEI hour
Th htas.IANorthimar wois.
VE: sury, alodr. foo Ro, ad we teltealtretor mousie Yougoifagater gorire, Sip  Sct..
LE: Calid tos Gere Theresnd.
OFFou's tthel, Thindit spepor IADReeme lle! yonou, othayonth  Ohe f har lld tis HA rodsanoulewhint FFIOhe grto, ICO: rods  t.
LEREERind y atholontllle YOfind s cer..
HAne.
Ithedin'tilt t lour. S: Coorfie t Bun..
LA: ithis, ldilay Wheewes, BETHANo s ADEEPu CEREIA: o.
HA: it ak. thnere  Ssict, THA: Can, We se.
LEEIO: t'side d t I afag fis.!
DE: Ye d


### Milestone 3: The mathematical trick in self-attention

In [26]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [24]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [25]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [30]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [31]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

In [34]:
# version 4: self-attention! every single nodes will emit 2 vectors: key & query, query vector what we look for, key: what we contain, weight = query @ key
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [35]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [36]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [37]:
k.var()

tensor(1.0449)

In [38]:
q.var()

tensor(1.0700)

In [39]:
wei.var()

tensor(1.0918)